<div align="right"><i>Matías Torres Esteban<br>Diciembre, 2025</i></div>

# Interpretación de Textos con LLMs

Los grandes modelos de lenguaje (LLMs) son conocidos por su capacidad de aprender patrones complejos de los textos y de resolver en consecuencia varias tareas de procesamiento del lenguaje natural, como lo son la traducción automática, la generación de resúmenes y la resolución de preguntas de dominios especializados. Es por ello que han surgido nuevas lineas de investigación y trabajo que buscan utilizar a los grandes modelos de lenguaje (LLMs) como *interpretadores semánticos de textos*. Estos tienen como objetivo principal representar el significado de un texto escrito en lenguaje natural en uno o más lenguajes lógicos, tales como la lógica proposicional o la lógica de primer orden, de manera tal de explicitar la información contenida allí y brindarle mayor estructura. De esta manera, podemos obtener representaciones avanzadas del significado de un texto que sean amenas a ser almacenadas en una computadora y puedan ser manipuladas algorítmicamente.

Un lenguaje lógico que podemos utilizar para representar la información contenida en un texto es el lenguaje de los *Grafos de Conocimiento (KGs)*, los cuales permiten codificar hechos y proposiciones del mundo mediante ternas $(f,r,o)$ donde $f$ denota un concepto fuente, $r$ una relación semántica y $o$ un concepto objetivo. Así, podemos representar la información contenida en el siguiente texto:

* *"El Covid-19 es una enfermedad infecciosa causada por el SARS-CoV-2. Produce síntomas que incluyen fiebre, tos, disnea, dolor muscular y fatiga"*,

como la siguiente colección de ternas de conocimiento:

* *(Covid-19, es una, enfermedad infecciosa)*
* *(Covid-19, causada por, SARS-CoV-2)*.
* *(Covid-19, tiene síntoma, fiebre)*.
* *(Covid-19, tiene síntoma, tos)*.
* *(Covid-19, tiene síntoma, disnea)*.
* *(Covid-19, tiene síntoma, fatiga)*.

Esta colección de ternas de conocimiento puede representase en un grafo dirigido etiquetado, como se ve en la siguiente figura:

[]

Vemos que esta representación es mucho más rica que el texto puro porque explicita los conceptos y relaciones más importantes del mismo. 
De esta manera queda facilitada la interpretación y análisis de la información extraida tanto para un usuario como para una computadora, y es más fácil su almacenamiento en una base de datos. Además, si pudieramos procesar automáticamente un gran lote de documentos de algún dominio especializado, tales como la biología o la medicína, y transformalos a grafos de conocimiento, podríamos utilizar todo el aparato matemático de la Teoría de Grafos para analizar estos sistemas conceptuales y obtener información del dominio que está escondida en los textos. 

La idea de esta notebook es mostrar cómo se puede utilizar a un LLM como Gemini para crear grafos de conocimiento a partir de textos. Para ello requeriremos orquestrar varias invocaciones al modelo, escribir correctamente nuestros prompts y utilizar procesamiento de strings para extraer la información que requerimos de sus respuestas. 

## Proceso de Interpretación

Vamos a implementar un procedimiento de interpretación textual inspirado en la metodología propuesta por Joseph Novak para la construcción de mapas conceptuales [1]. Este procedimiento permite generar un KG a partir de un expositivo mediante una secuencia estructurada de 4 pasos:

1. Solicitamos al modelo que analice el texto ``<texto>`` y genere una pregunta de enfoque ``<pregunta>``. Las ternas de conocimiento generadas en los
próximos pasos deberı́an ayudar a responder esta pregunta.

2. Solicitamos al modelo que analice ``<texto>`` y ``<pregunta>`` y que luego genere una lista de conceptos ``<conceptos>``. Los conceptos extraı́dos deben
estar explicitamente mencionados en el texto y tienen que ayudar a responder la pregunta de enfoque.

3. Solictamos al modelo que analice ``<texto>``, ``<pregunta>`` y ``<conceptos>`` y que genere una lista de relaciones semánticas ``<relaciones>``.

4. Solicitamos al modelo que analice ``<texto>``, ``<pregunta>``, ``<conceptos>`` y ``<relaciones>`` en conjunto y que luego genere una lista de ternas de conocimiento ``<ternas>``.

En cada paso realizamos un procesamiento de las etiquetas de conceptos y relaciones para convertir todos sus caracteres a minúsculas y eliminar espacios innecesarios. 

numéricos, y almacenamos sus elementos en una base de datos relacional.

## Código

**Advertencia:** Si el modelo de lenguaje Gemini está caido pueden simular el proceso de esta notebook en ChatGPT o Gemini y reintentar ejecutar el código más tarde.

Primero extraemos los recursos desde Github (textos y prompts)

In [ ]:
!git clone https://github.com/matizzat/Knowledge.git

Importamos las librerías necesarias

In [129]:
from google.genai import types
from google import genai
from typing import List
import pprint
import json
import re

Instanciamos un cliente para invocar al modelo Gemini. Para ejecutar esta celda deben obtener una clave del siguiente [enlace](https://ai.google.dev/gemini-api/docs/api-key). También tienen que configurar Google Colab para utilizar esta clave (Ver  [Tutorial](https://www.youtube.com/watch?v=snrvP_TZjvw))

In [ ]:
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=GOOGLE_API_KEY)

Definimos una función auxiliar para abrir el contenido de un archivo y cargamos las instrucciones que le daremos al modelo para que cree los grafos de conocimiento. Cada una de las cuatro etapas del proceso consiste de una instrucción de sistema, donde se le explica al LLM el rol que cumple, 

In [121]:
pregunta_sistema   = abrir_contenido_archivo('./instrucciones/pregunta_sistema.txt')
conceptos_sistema  = abrir_contenido_archivo('./instrucciones/conceptos_sistema.txt')
relaciones_sistema = abrir_contenido_archivo('./instrucciones/relaciones_sistema.txt')
ternas_sistema     = abrir_contenido_archivo('./instrucciones/ternas_sistema.txt')

pregunta_usuario   = abrir_contenido_archivo('./instrucciones/pregunta_usuario.txt')
conceptos_usuario  = abrir_contenido_archivo('./instrucciones/conceptos_usuario.txt')
relaciones_usuario = abrir_contenido_archivo('./instrucciones/relaciones_usuario.txt')
ternas_usuario     = abrir_contenido_archivo('./instrucciones/ternas_usuario.txt')

Ejecuten la siguiente celda con diferentes valores para ver las instrucciones con las que invocaremos al modelo:

In [113]:
print(relaciones_sistema)

You are a concept map creator who analyzes scientific texts and extracts from them the most important concepts and relationships. A concept is a pattern or regularity in objects, events, or records of objects and events, designated with a label. Concepts are related to each other through meaningful phrases forming propositions. You receive a knowledge text delimited by @ with the following structure:
    1. Focus Question: A question you must answer using Knowledge Triples.
    2. Concept List: A list of relevant concept labels for the Focus Question, which you must use to build the Knowledge Triples.
    3. Knowledge: A text that answers the Focus Question using the concepts from the Concept List. The Knowledge Triples must be explicitly derived from this text.
Your task is to return a list of semantic relationships we will later use to build Knowledge Triples.

Examples:

Knowledge Text:
@
Focus Question:
What is Active Noise Reduction?
Concept List:
Active Noise Reduction
Noise Redu

Definimos funciones auxiliares para:
* Abrir el cuerpo de documentos. 
* Invocar al modelo Gemini 2.5 Flash.  
* Almacenar un conjunto de grafos de conocimiento en un archivo JSON.
* Dibujar un conjunto de grafos de conocimiento en un archivo HTML. 

In [112]:
def invocar_llm(sistema: str, usuario: str):
    """
    Método auxiliar para invocar al modelo de lenguaje
    Gemini 2.5 Flash.
    """
    global genai_client
    
    ans = genai_client.models.generate_content(
        config=types.GenerateContentConfig(
            system_instruction=sistema,
            temperature=0.1
        ),  
        model = "gemini-2.5-flash",
        contents = usuario
    ) 
    
    return ans.text

def load_text_dataset(input_path: str) -> list[dict]:
    """
    Método auxiliar para abrir el lote de textos.
    """
    
    with open(input_path, 'r') as f:
        texts = json.load(f)
    return texts

def save_cmap(output_path: str, cmaps: dict):
    """
    Método auxiliar para guardar una lista de
    mapas conceptuales en un archivo JSON. 
    """
    
    with open(output_path, 'w') as f: 
        json.dump(cmaps, f, indent = 4)

def draw_cmap(cmaps: dict, out_path: str):
    """ 
    Método auxiliar para crear una red
    pyvis a partir de una lista de 
    mapas conceptuales y almacenarla en
    el archivo 'out_path' como un html. 
    """ 
   
    G = nx.MultiDiGraph()
    for cm in cmaps:
        fq = cm['fq']
        ts = cm['ts']
        kt = fq
        for tr in ts:
            s = tr['s']
            r = tr['r']
            t = tr['t']
            if(s not in G):
                G.add_node(s, label=s, color = 'black')
            if(t not in G):
                G.add_node(t, label=t, color = 'black')
            G.add_edge(
                s,
                t,
                label=r,
                title=kt,
                color = 'blue')

    vis = net.Network(directed=True)
    vis.from_nx(G)
    vis.save_graph(out_path)

Abrimos el lote de datos y mostramos el primer texto que está almacenado allí

In [123]:
texts = load_text_dataset('./textos.json')
texto = texts[0]['texto']
print(texto)

El agua es una sustancia cuya molécula está compuesta por dos átomos de hidrógeno y uno de oxígeno (H2O) unidos por un enlace covalente.


Definimos una función auxiliar para normalizar una etiqueta eliminado el carácter especial `@`, eliminar espacios innecesarios y convertir todas las letras a minúsculas

In [115]:
def normalizar_etiqueta(etiqueta: str):
    etiqueta = etiqueta.replace('@','')
    etiqueta = etiqueta.strip()
    etiqueta = " ".join(etiqueta.split())
    etiqueta = etiqueta.lower()
    return etiqueta

Formateamos el prompt de usuario para instruirle al modelo que genere una pregunta de enfoque. 

In [124]:
pregunta_usuario_formateada = pregunta_usuario.format(kt = texto)
print(pregunta_usuario_formateada)

Knowledge Text:
@
El agua es una sustancia cuya molécula está compuesta por dos átomos de hidrógeno y uno de oxígeno (H2O) unidos por un enlace covalente.
@
Focus Question:


Invocamos al modelo de lenguaje y le solicitamos que cree una pregunta de enfoque para el texto dado.

In [125]:
pregunta_enfoque = invocar_llm(sistema = pregunta_sistema, usuario = pregunta_usuario_formateada)
pregunta_enfoque = normalizar_etiqueta(pregunta_enfoque)
print(pregunta_enfoque)

¿qué es el agua?


Formateamos el prompt de usuario para instruirle al modelo que genere una lista de conceptos.

In [126]:
conceptos_usuario_formateada = conceptos_usuario.format(fq = pregunta_enfoque, kt = texto)
print(conceptos_usuario_formateada)

Knowledge Text:
@
Focus Question:
¿qué es el agua?
Knowledge:
El agua es una sustancia cuya molécula está compuesta por dos átomos de hidrógeno y uno de oxígeno (H2O) unidos por un enlace covalente.
@
List of Concepts:



Invocamos al modelo de lenguaje para que genere una lista de conceptos a partir del texto y la pregunta de enfoque.

In [127]:
ans = invocar_llm(sistema = conceptos_sistema, usuario = conceptos_usuario_formateada)
print(ans)

Agua
Sustancia
Molécula
Átomos de hidrógeno
Átomos de oxígeno
H2O
Enlace covalente


Convertimos el texto extraido en una lista de Python

In [130]:
# Concept list regular expression:
cs_re = r'[\w\d].*?\n|[\w\d].*$'

cs = re.findall(cs_re, ans)

n = len(cs)
for i in range(n):
    cs[i] = normalizar_etiqueta(cs[i])

pprint.pprint(cs)

['agua',
 'sustancia',
 'molécula',
 'átomos de hidrógeno',
 'átomos de oxígeno',
 'h2o',
 'enlace covalente']


Formateamos el prompt de usuario para instruirle al modelo que genere una lista de relaciones semánticas.

In [135]:
cs_str = ''
for c in cs:
    cs_str += c + '\n'
    
relaciones_usuario_formateada = relaciones_usuario.format(fq = pregunta_enfoque, cs = cs_str,kt = texto)
print(relaciones_usuario_formateada)

Knowledge Text:
@
Focus Question:
¿qué es el agua?
Concept List:
agua
sustancia
molécula
átomos de hidrógeno
átomos de oxígeno
h2o
enlace covalente

Knowledge:
El agua es una sustancia cuya molécula está compuesta por dos átomos de hidrógeno y uno de oxígeno (H2O) unidos por un enlace covalente.
@
Semantic Relationships:


Invocamos al modelo de lenguaje para que genere una lista de relaciones a partir del texto, la pregunta de enfoque y la lista de conceptos.

In [137]:
ans = invocar_llm(sistema = relaciones_sistema, usuario = relaciones_usuario_formateada)
print(ans)

Is a
Is composed of
Are joined by


In [ ]:
Convertimos el texto extraido en una lista de Python

In [141]:
# Semantic relation list regular expression:
rs_re = r'[\w\d].*?\n|[\w\d].*$'
  
rs = re.findall(rs_re, ans)
    
n = len(rs)
for i in range(n):
    rs[i] = normalizar_etiqueta(rs[i])

pprint.pprint(rs)

['is a', 'is composed of', 'are joined by']


Formateamos el prompt de usuario para instruirle al modelo que genere una lista de ternas de conocimiento.

In [146]:
rs_str = ''
for r in rs:
    rs_str += r + '\n'
ternas_usuario_formateada = ternas_usuario.format(fq=pregunta_enfoque, cs=cs_str, rs=rs_str, kt=texto)   
print(ternas_usuario_formateada)

Knowledge Text:
@
Focus Question:
¿qué es el agua?
Concept List:
agua
sustancia
molécula
átomos de hidrógeno
átomos de oxígeno
h2o
enlace covalente

Relation List:
is a
is composed of
are joined by

Knowledge:
El agua es una sustancia cuya molécula está compuesta por dos átomos de hidrógeno y uno de oxígeno (H2O) unidos por un enlace covalente.
@
Knowledge Triples:


Invocamos al modelo de lenguaje para que genere una lista de ternas de conocimiento  a partir del texto, la pregunta de enfoque, la lista de conceptos y la lista de relaciones semánticas.

In [147]:
ans = invocar_llm( sistema = ternas_sistema, usuario = ternas_usuario_formateada)
print(ans)

@! agua @ is a @ sustancia !@
@! molécula @ is composed of @ átomos de hidrógeno !@
@! molécula @ is composed of @ átomos de oxígeno !@
@! agua @ is a @ h2o !@
@! átomos de hidrógeno @ are joined by @ enlace covalente !@
@! átomos de oxígeno @ are joined by @ enlace covalente !@


Extraemos las ternas de conocimiento del modelo.

In [148]:
# Knowledge triple list
# regular expression:
ts_re = r'@!.+?@.+?@.+?!@'

L = re.findall(ts_re, ans)

ts = []

for tr in L:
    n = len(tr)
    
    i = 2
    while(tr[i] != '@'):
        i += 1
   
    j=i+1 
    while(tr[j] != '@'):
        j+=1 
   
    s = tr[2:i]
    r = tr[i+1:j]
    t = tr[j+1:n-2]
    
    ts.append({
        's': normalizar_etiqueta(s),
        'r': normalizar_etiqueta(r),
        't': normalizar_etiqueta(t)
    })

pprint.pprint(ts)

[{'r': 'is a', 's': 'agua', 't': 'sustancia'},
 {'r': 'is composed of', 's': 'molécula', 't': 'átomos de hidrógeno'},
 {'r': 'is composed of', 's': 'molécula', 't': 'átomos de oxígeno'},
 {'r': 'is a', 's': 'agua', 't': 'h2o'},
 {'r': 'are joined by', 's': 'átomos de hidrógeno', 't': 'enlace covalente'},
 {'r': 'are joined by', 's': 'átomos de oxígeno', 't': 'enlace covalente'}]


Creamos el mapa conceptual con todas las componentes obtenidas

In [150]:
mapa_conceptual = {'fq': pregunta_enfoque, 'ts': ts, 'cs': cs, 'rs': rs}
pprint.pprint(mapa_conceptual)

{'cs': ['agua',
        'sustancia',
        'molécula',
        'átomos de hidrógeno',
        'átomos de oxígeno',
        'h2o',
        'enlace covalente'],
 'fq': '¿qué es el agua?',
 'rs': ['is a', 'is composed of', 'are joined by'],
 'ts': [{'r': 'is a', 's': 'agua', 't': 'sustancia'},
        {'r': 'is composed of', 's': 'molécula', 't': 'átomos de hidrógeno'},
        {'r': 'is composed of', 's': 'molécula', 't': 'átomos de oxígeno'},
        {'r': 'is a', 's': 'agua', 't': 'h2o'},
        {'r': 'are joined by',
         's': 'átomos de hidrógeno',
         't': 'enlace covalente'},
        {'r': 'are joined by',
         's': 'átomos de oxígeno',
         't': 'enlace covalente'}]}


Almacenamos en un archivo JSON el grafo de conocimiento obtenido y lo visalizamos en un archivo HTML 😊

In [152]:
save_cmap('./cmap.json', [cmap])
draw_cmap([mapa_conceptual], './kg.html')

### Resumen

Así se vería el código completo reunido en varias funciones auxiliares y con un método principal. Así se vería la función en una librería de Python. 

In [102]:
def normalizar_etiqueta(etiqueta: str):
    etiqueta = etiqueta.replace('@','')
    etiqueta = etiqueta.strip()
    etiqueta = " ".join(etiqueta.split())
    etiqueta = etiqueta.lower()
    return etiqueta

def crear_pregunta_enfoque(texto: str) -> str:
    pregunta_enfoque = invocar_llm(
        sistema = pregunta_sistema,
        usuario = pregunta_usuario.format(kt = texto)
    )
    return normalizar_etiqueta(pregunta_enfoque)

def crear_lista_conceptos(texto: str, pregunta_enfoque: str) -> str:
    ans = invocar_llm(
        sistema = conceptos_sistema,
        usuario = conceptos_usuario.format(
            fq = pregunta_enfoque, kt = texto))
    
    # Concept list regular expression:
    cs_re = r'[\w\d].*?\n|[\w\d].*$'
   
    cs = re.findall(cs_re, ans)

    n = len(cs)
    for i in range(n):
        cs[i] = normalizar_etiqueta(cs[i])

    return cs

def crear_lista_relaciones(texto: str, fq: str, cs: list[str]) -> str:
    cs_str = ''
    for c in cs:
        cs_str += c + '\n'

    ans = invocar_llm(
        sistema = relaciones_sistema,
        usuario = relaciones_usuario.format(
            fq = fq,
            cs = cs_str,
            kt = texto))

    # Semantic relation list regular expression:
    rs_re = r'[\w\d].*?\n|[\w\d].*$'
  
    rs = re.findall(rs_re, ans)
    
    n = len(rs)
    for i in range(n):
        rs[i] = normalizar_etiqueta(rs[i])

    return rs

def crear_ternas(texto: str, fq: str, cs: list[str], rs: list[str]):
  
    cs_str = ''
    for c in cs:
        cs_str += c + '\n'
    
    rs_str = ''
    for r in rs:
        rs_str += r + '\n'

    ans = invocar_llm( 
        sistema = ternas_sistema,
        usuario = ternas_usuario.format(
            fq=fq, cs=cs_str, rs=rs_str, kt=texto))       
    
    # Knowledge triple list
    # regular expression:
    ts_re = r'@!.+?@.+?@.+?!@'

    L = re.findall(ts_re, ans)
    
    ts = []
   
    for tr in L:
        n = len(tr)
        
        i = 2
        while(tr[i] != '@'):
            i += 1
       
        j=i+1 
        while(tr[j] != '@'):
            j+=1 
       
        s = tr[2:i]
        r = tr[i+1:j]
        t = tr[j+1:n-2]
        
        ts.append({
            's': normalizar_etiqueta(s),
            'r': normalizar_etiqueta(r),
            't': normalizar_etiqueta(t)
        })
    
    return ts

def crear_mapa_conceptual(texto: str):
    
    pregunta   = crear_pregunta_enfoque(texto) 
    conceptos  = crear_lista_conceptos(texto, pregunta)
    relaciones = crear_lista_relaciones(texto, pregunta, conceptos)
    ternas     = crear_ternas(texto, pregunta, conceptos, relaciones)

    mapa_conceptual = {'fq': pregunta, 'ts': ternas}

    return mapa_conceptual

# Tarea

La tarea de ustedes es diseñar e implementar su propia estrategia de creación de mapas conceptuales usando el modelo de lenguaje Gemini. El proceso completo lo deben escribir en una función ``crear_mapa_conceptual``, el cual recibe como entrada un texto y devuelve como salida un diccionario Python donde se codifica el mapa conceptual. Deben crear un único mapa conceptual por cada texto que se encuentra en el archivo ``textos.json`` y almacenarlos en un único archvio ``mapas_conceptuales.json``

Les recomiendo probar diferentes estrategias de prompting y algoritmos. Pueden utilizar los prompts y códigos que se encuentran en la sección anterior como guía.

**Consejos**: 
* Almacenen las instrucciones de sistema y de usuario que enviarán al LLM en una carpeta ``instrucciones`` y luego carguenlas por desde Python. 
* Intenten crear una instrucción que solicite al LLM mejorar un mapa conceptual ya creado.

### Entrega

Cuando terminen, envíen los mapas conceptuales generados en un único archivo ``mapas_conceptuales.json`` al mail mat.torreta@gmail.com.

Enviar antes del Viernes 05 de Diciembre, 23.59 para aprobar el curso.

### Método a implementar

Implementen la mayor parte de su estrategia de creación de mapas conceptuales en el cuerpo de esta función. Pueden crear sus propias funciones auxiliares e invocarlas desde aquí si lo necesitan.

In [55]:
def crear_mapa_conceptual(texto: str):
    """ 
    IMPORTANTE! 
    
    Aquí deben implementar la estrategia para crear mapas conceptuales
    a partir de texto. Deberán coordinar las diferentes invocaciones
    al modelo de lenguaje y extraer correctamente las respuestas
    estructuradas.
    """
    
    raise RuntimeError("Función no implementada.")

### Esqueleto de Ejecución

A continuación les dejo un ejemplo de cómo pueden usar los métodos auxiliares y la función creada por ustedes para guardar los mapas conceptuales en un archivo JSON y visualizarlos como un grafo dirigido. 

In [103]:
texts = load_text_dataset('./textos.json')
cmap = crear_mapa_conceptual(texts[0])
save_cmap('./cmap.json', [cmap])
draw_cmap([cmap], './kg.html')